# Generating User Facts

### Features that pertain to users dietary preferences are dairyOpinion, meatOpinion, nutsOpinion.
### For each feature, 33% of the users will have an opinion of 1(love), 33% of the users will have an opinion of 0 (neutral),
### and the remaining 33% of the users will have an opinion of -1(hate)

In [1]:
import pandas as pd
import random
import os
import json

In [2]:
users = list(range(1, 26))
user_facts = []
food_facts = []
pos_pairs = []
neg_pairs = []

In [3]:
def partition(arr):
    # Shuffle the array
    random.shuffle(arr)
    
    # Determine the size of each partition
    partition_size = len(arr) // 3
    
    # Create the three partitions
    part1 = arr[:partition_size]
    part2 = arr[partition_size:partition_size * 2]
    part3 = arr[partition_size*2:]
    
    return part1, part2, part3

In [4]:
dairy_opinions = partition(users)
meat_opinions = partition(users)
nut_opinions = partition(users)

for feature_name, opinions in [('dairy',dairy_opinions),
                               ('meat', meat_opinions), 
                               ('nuts', nut_opinions)]:
    positive, negative, _ = opinions
    for opinion, type_str in [(positive,'positive'),
                           (negative, 'negative')]:
        for user in opinion:
            user_facts.append(f'preference(user_{user}, {type_str}_{feature_name}).')

# Generating Food Facts

In [5]:
# Loading data
beverages = json.load(open('../items_data/beverages.json'))
beverages = beverages['beverage_ids']

mcdonalds = json.load(open('../items_data/mcdonalds.json'))
mcdonalds = mcdonalds['recipe-ids']

taco_bell = json.load(open('../items_data/taco_bell.json'))
taco_bell = taco_bell['recipe-ids']

treat_data=json.load(open('../items_data/recipe_repn.json'))
treat_data = treat_data['recipe-ids']

In [6]:
for data, is_bev in ((beverages, True), (mcdonalds, False), (taco_bell, False), (treat_data, False)):
    for key, item_info in data.items():
        if item_info['hasNuts']:
            if is_bev:
                food_facts.append(f'item(bev_{key}, has_nuts).')
            else:
                food_facts.append(f'item(food_{key}, has_nuts).')
        if item_info['hasMeat']:
            if is_bev:
                food_facts.append(f'item(bev_{key}, has_meat).')
            else:
                food_facts.append(f'item(food_{key}, has_meat).')
        if item_info['hasDairy']:
            if is_bev:
                food_facts.append(f'item(bev_{key}, has_dairy).')
            else:
                food_facts.append(f'item(food_{key}, has_dairy).')

# Generating positive/negative pairs

In [7]:
pos_dairy, neg_dairy, _ = dairy_opinions
pos_meat, neg_meat, _ = meat_opinions
pos_nuts, neg_nuts, _ = nut_opinions

pos_scores = 0
neg_scores = 0
zero_scores = 0

for user in users:
    for data, is_bev in ((beverages, True), (mcdonalds, False), (taco_bell, False), (treat_data, False)):
        for key, item_info in data.items():
            score = 0
            if ((item_info['hasNuts'] and user in neg_nuts) or
                 (item_info['hasDairy'] and user in neg_dairy) or
                 (item_info['hasMeat'] and user in neg_meat)):
                if is_bev:
                    neg_pairs.append(f'recommendation(user_{user}, bev_{key}).')
                else:
                    neg_pairs.append(f'recommendation(user_{user}, food_{key}).')
            else:
                if is_bev:
                    pos_pairs.append(f'recommendation(user_{user}, bev_{key}).')
                else:
                    pos_pairs.append(f'recommendation(user_{user}, food_{key}).')

# Splitting Train/Test

In [8]:
def split_train_test(array, per_train=0.8):
    # shuffle the array
    random.shuffle(array)

    # Determine the split index
    split_index = int(per_train * len(array))
    
    # Split the array into train test
    train = array[:split_index]
    test = array[split_index:]
    return train, test

In [9]:
user_train, user_test = split_train_test(user_facts) 
food_train, food_test = split_train_test(food_facts) 
train_pos, test_pos = split_train_test(pos_pairs) 
train_neg, test_neg = split_train_test(neg_pairs) 

In [10]:
train_facts = user_train + food_train
test_facts = user_test + food_test

In [12]:
for contents, file_name in [(train_facts, 'train_facts.txt'),
                            (train_neg, 'train_neg.txt'),
                            (train_pos, 'train_pos.txt'),
                            (test_facts, 'test_facts.txt'),
                            (test_neg, 'test_neg.txt'),
                            (test_pos, 'test_pos.txt')]:
    with open(f'../boosted_bandit/{file_name}', 'w') as file:
        contents = [(line + '\n') for line in contents]
        file.writelines(contents)